<a href="https://colab.research.google.com/github/s-sasaki-gunma/MaterialApp/blob/main/App_HuffmanTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ハフマン符号化とは？**
- 文章中の文字の出現頻度の偏りを利用した圧縮技術のこと
- よく出現する文字には短いビット列を、あまり出現しない文字には長いビット列を割り当てることで、メッセージ全体の符号化に使われるデータ量を削減する

## ハフマン符号化による圧縮の手順
1. 文章中の各文字の出現回数を数える
2. 出現回数に応じてハフマン木を作る
   0. 文字の種類だけ節点を用意する
   1. 出現回数順に節点を並び替える
   2. １番小さい節点と次に小さい節点を選び、新しい節点を作り枝を結ぶ
   3. すべての節点がつながるまで手順2-2を繰り返す
   4. 木の各節点の枝の左右に0と1を割り当てる
4. 一番上の節点から順に0と1を辿り、各文字の二進符号として割り当てる
5. 元の文章の各文字を二進符号に置換する（圧縮完了）

In [ ]:
# @title ★ 文字列「A A B A A A E C A A B A D A A A B C」の出現回数の表を作り、ハフマン木を描きます。<br>state_stepを0から増やしてハフマン木を描く手順を確認しましょう。 { run: "auto" }
state_step = 0 # @param {type:"slider", min:0, max:100, step:1}

import math
import random
import pandas as pd
import collections
import graphviz
from graphviz import Digraph
from IPython.display import Image, display_png

class Node:
    def __init__(self, name, value, left, right, weight):
        self.NAME = name
        self.V = value
        self.L = left
        self.R = right
        self.W = weight
        self.CODE = None

    def calc(self, code):
        self.CODE = code[1:]
        if self.L and self.R  :
            self.L.calc( code + str(0) )
            self.R.calc( code + str(1) )

    def getCSV(self,ary):
        if self.L and self.R  :
            ary.append([ary.append(self.L.getCSV( ary )), ary.append(self.R.getCSV( ary ))])
        else:
            return [self.NAME, self.V, self.CODE, self.W]

    def getEndNodeCODEs(self):
        if self.L and self.R  :
            return self.L.getEndNodeCODEs() + "," + self.R.getEndNodeCODEs()
        else :
            return self.CODE

    def getFormula(self):
        if self.L and self.R  :
            return self.L.getFormula() + " + " + self.R.getFormula()
        else :
            return str(self.V) + " [文字] x " + str(len(self.CODE)) + " [bit]"

    def getDataSize(self):
        if self.L and self.R  :
            return self.L.getDataSize() + self.R.getDataSize()
        else :
            return self.V * len( self.CODE )

def sentence2dict(sentence):
    s={}
    N=0
    if sentence == "" :
        txt="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        N = random.randint(3,10)
        for i in range(N):
            s.update( { txt[i] : random.randint(1,100)} )
    else :
        t = list(sentence.replace(' ', '').replace('　', '').replace('\t', '').replace('\n', ''))
        txt = sorted(list(set(t)))
        N = len(txt)
        for i in range(N) :
            s.update( { txt[i] : t.count(txt[i]) } )
    return s , N

def HuffmanTree(dct):
    n=[]
    dct=sorted( dct.items() , key=lambda x:x[1] )
    for i in dct :
        n.append( Node( i[0], i[1], None, None, None ) )
    W = 0
    while len(n) > 1 :
        if len(n) == 2 :
            p = 0
        elif len(n) == 3 :
            if n[0].V + n[1].V <= n[1].V + n[2].V :
                p = 0
            else :
                p = 1
        else :
            p = n.index(min(n, key=lambda x:(x.V)))
            if p>0 and n[p-1].V + n[p].V <= n[p].V + n[p+1].V :
                p -= 1
        R = n.pop(p)
        L = n.pop(p)
        V = R.V + L.V
        W += 1
        n.insert( p, Node( None , V , L , R , W) )
    n[0].calc("_")
    tmp=[]
    n[0].getCSV(tmp)
    return n[0]

def preorder(g, n):
    if n:
        if n.L and n.R :
            if n.W <= state_step :
                g.node( n.CODE, label=str(n.V) )
                if N <= state_step :
                    g.edge( n.CODE, n.CODE+str(0), label=str(0) )
                else :
                    g.edge( n.CODE, n.CODE+str(0) )
            preorder( g, n.L )
            if n.W <= state_step :
                if N <= state_step :
                    g.edge( n.CODE, n.CODE+str(1), label=str(1) )
                else :
                    g.edge( n.CODE, n.CODE+str(1) )
            preorder( g, n.R )
        else :
            with g.subgraph(name='cluster_'+n.CODE) as sg:
                if N+1 <= state_step :
                    sg.node(n.CODE, label="{{%s|%s}|%s}" % ( str(n.NAME), str(n.V), n.CODE ), shape='Mrecord', color='#FF0000', fontcolor='#FF0000')
                else :
                    sg.node(n.CODE, label="%s|%s" % ( str(n.NAME), str(n.V) ), shape='Mrecord', color='#FF0000', fontcolor='#FF0000')

def setGraphRank(g, n):
    endNL = n.getEndNodeCODEs().split()
    attr = "{rank=same; "
    for i in endNL:
        attr += i + "; "
    attr += "}"
    g.body.extend(attr)

if __name__ == "__main__":
    sentence = "A A B A A A E C A A B A D A A A B C"
    S,N = sentence2dict(sentence)
    print("文字の出現回数の表")
    display(pd.DataFrame(data=S, index=["文字数"]))
    G = Digraph(format='png')
    G.attr('graph',rankdir='TB')
    G.attr('node', shape='circle')
    node = HuffmanTree(S)
    preorder(G, node)
    setGraphRank(G, node)
    print("\nハフマン木")
    G.render('binary_tree', view=True)
    display_png(Image('binary_tree.png'))

## ★ 文字列「A A B A A A E C A A B A D A A A B C」のデータ量の計算
### 非圧縮（固定長）
- 文章中の文字の種類が $5$ 種のため、$log_2(5)=2.32...$ より１文字あたり最低でも３桁の二進数で表す必要がある。

|  | A | B | C | D | E |
|:---:|:---:|:---:|:---:|:---:|:---:|
|二進符号| 000 | 001 | 010 | 011 | 100 |

- 文字列「A A B A A A E C A A B A D A A A B C」を２進数に変換すると、<br>
000 000 001 000 000 000 100 010 000 000 001 000 011 000 000 000 001 010
- 54桁の二進数で表現できるため、データ量は $54bit$ である

### ハフマン符号化による圧縮
- ハフマン木より出現回数の多い文字に短い二進数を、少ない文字列に長い二進数を割り当てる

|  | A | B | C | D | E |
|:---:|:---:|:---:|:---:|:---:|:---:|
|二進符号| 0 | 10 | 110 | 1110 | 1111 |

- 文字列「A A B A A A E C A A B A D A A A B C」を２進数に変換すると、<br>
0 0 10 0 0 0 1111 110 0 0 10 0 1110 0 0 0 10 110
- 31桁の二進数に表現できるため、データ量は $31bit$ である
- 非圧縮のときに比べて $54bit$ から $31bit$ へとデータ量を削減でき、その圧縮率は $\dfrac{31 [bit]}{54 [bit]}=57.4 [％] $ である
<br>
<br>

# **ハフマン符号化による文字列の圧縮を確認しましょう**
- 次の①から④までのコードを上から順番に実行すると確認できます
- 何回か実行して文字の出現回数の偏りによりハフマン木の形状が異なることを確認しよう
- ①で自動生成した文字の出現回数の表をもとにハフマン木を自分で書く練習をしてみよう

In [ ]:
# @title ①　各文字の出現回数を数えます。ハフマン木にしたい文章をsentenceの横に入力してから実行ください。<br>　（空白のまま実行するとランダムな文字の出現回数の表を用意します。） { display-mode: "form" }
sentence = "" # @param ["A A B A A A E C A A B A D A A A B C", "A A D A D A D A D B D D A B A E E C E A A A E A A D A A A", "A C A A C C D D D C A A E C C A A A B D B B E B B E D B E B", "A C C C D E E D E C C A B A A B B B A B B A A C C C A A A A A B B B B D D"] {allow-input: true}

import math
import random
import pandas as pd
import collections

def sentence2dict(sentence):
    s={}
    N=0
    if sentence == "" :
        txt="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        N = random.randint(3,10)
        for i in range(N):
            s.update( { txt[i] : random.randint(1,100)} )
    else :
        t = list(sentence.replace(' ', '').replace('　', '').replace('\t', '').replace('\n', ''))
        txt = sorted(list(set(t)))
        N = len(txt)
        for i in range(N) :
            s.update( { txt[i] : t.count(txt[i]) } )
    return s , N

if __name__ == "__main__":
#    sentence = input("入力欄：")
    S,N = sentence2dict(sentence)
    display(pd.DataFrame(data=S, index=["文字数"]))

In [ ]:
# @title ②　圧縮前のデータ量を計算します。
bit = math.ceil(math.log2(N))
total = sum(S.values())
print("１文字あたり　：",bit, "[bit]  ≧ log2(%s)" % N)
print("文字数合計　　：",total, "[文字]")
print("圧縮前データ量：",total*bit, "[bit]")

In [ ]:
# @title ③　ハフマン木を表示します。state_stepを0から順に増やして、木の構築手順を確認しましょう。 { run: "auto" }
state_step = 0 # @param {type:"slider", min:0, max:100, step:1}

import graphviz
from graphviz import Digraph
from IPython.display import Image, display_png

class Node:
    def __init__(self, name, value, left, right, weight):
        self.NAME = name
        self.V = value
        self.L = left
        self.R = right
        self.W = weight
        self.CODE = None

    def calc(self, code):
        self.CODE = code[1:]
        if self.L and self.R  :
            self.L.calc( code + str(0) )
            self.R.calc( code + str(1) )

    def getCSV(self,ary):
        if self.L and self.R  :
            ary.append([ary.append(self.L.getCSV( ary )), ary.append(self.R.getCSV( ary ))])
        else:
            return [self.NAME, self.V, self.CODE, self.W]

    def getEndNodeCODEs(self):
        if self.L and self.R  :
            return self.L.getEndNodeCODEs() + "," + self.R.getEndNodeCODEs()
        else :
            return self.CODE

    def getFormula(self):
        if self.L and self.R  :
            return self.L.getFormula() + " + " + self.R.getFormula()
        else :
            return str(self.V) + " [文字] x " + str(len(self.CODE)) + " [bit]"

    def getDataSize(self):
        if self.L and self.R  :
            return self.L.getDataSize() + self.R.getDataSize()
        else :
            return self.V * len( self.CODE )

def HuffmanTree(dct):
    n=[]
    dct=sorted( dct.items() , key=lambda x:x[1] )
    for i in dct :
        n.append( Node( i[0], i[1], None, None, None ) )
    W = 0
    while len(n) > 1 :
        if len(n) == 2 :
            p = 0
        elif len(n) == 3 :
            if n[0].V + n[1].V <= n[1].V + n[2].V :
                p = 0
            else :
                p = 1
        else :
            p = n.index(min(n, key=lambda x:(x.V)))
            if p>0 and n[p-1].V + n[p].V <= n[p].V + n[p+1].V :
                p -= 1
        R = n.pop(p)
        L = n.pop(p)
        V = R.V + L.V
        W += 1
        n.insert( p, Node( None , V , L , R , W) )
    n[0].calc("_")
    tmp=[]
    n[0].getCSV(tmp)
    return n[0]

def preorder(g, n):
    if n:
        if n.L and n.R :
            if n.W <= state_step :
                g.node( n.CODE, label=str(n.V) )
                if N <= state_step :
                    g.edge( n.CODE, n.CODE+str(0), label=str(0) )
                else :
                    g.edge( n.CODE, n.CODE+str(0) )
            preorder( g, n.L )
            if n.W <= state_step :
                if N <= state_step :
                    g.edge( n.CODE, n.CODE+str(1), label=str(1) )
                else :
                    g.edge( n.CODE, n.CODE+str(1) )
            preorder( g, n.R )
        else :
            with g.subgraph(name='cluster_'+n.CODE) as sg:
                if N+1 <= state_step :
                    sg.node(n.CODE, label="{{%s|%s}|%s}" % ( str(n.NAME), str(n.V), n.CODE ), shape='Mrecord', color='#FF0000', fontcolor='#FF0000')
                else :
                    sg.node(n.CODE, label="%s|%s" % ( str(n.NAME), str(n.V) ), shape='Mrecord', color='#FF0000', fontcolor='#FF0000')

def setGraphRank(g, n):
    endNL = n.getEndNodeCODEs().split()
    attr = "{rank=same; "
    for i in endNL:
        attr += i + "; "
    attr += "}"
    g.body.extend(attr)

if __name__ == "__main__":
    G = Digraph(format='png')
    G.attr('graph',rankdir='TB')
    G.attr('node', shape='circle')
    node = HuffmanTree(S)
    preorder(G, node)
    setGraphRank(G, node)
    G.render('binary_tree'+str(state_step).zfill(2), view=True)
    display_png(Image('binary_tree'+str(state_step).zfill(2)+'.png'))

In [ ]:
# @title ④　圧縮後のデータ量および圧縮率を計算する
bit = math.ceil(math.log2(N))
total = sum(S.values())
before_compress = total * bit
after_compress = node.getDataSize()
compressibility = after_compress / before_compress
print("圧縮前のデータ量 = ", total, "[文字] x", bit, "[bit]")
print("圧縮前：", before_compress, "[bit]")
print()
print("圧縮後のデータ量 = ",node.getFormula())
print("圧縮後：", after_compress, "[bit]")
print()
print("圧縮率：", round(compressibility*100,2), "%")